In [1]:
import ifcopenshell
import ifcopenshell.util.element
import ifcopenshell.guid

# Fonction pour ajouter un PropertySet

def add_property_set(ifc, element, name, properties):
    props = []
    for key, value in properties.items():
        if isinstance(value, (int, float)):
            nominal = ifc.create_entity("IfcPropertySingleValue", Name=key, NominalValue=ifc.create_entity("IfcReal", value))
        else:
            nominal = ifc.create_entity("IfcPropertySingleValue", Name=key, NominalValue=ifc.create_entity("IfcLabel", value))
        props.append(nominal)

    pset = ifc.create_entity("IfcPropertySet", GlobalId=ifcopenshell.guid.new(), Name=name, HasProperties=props)
    rel = ifc.create_entity("IfcRelDefinesByProperties",
        GlobalId=ifcopenshell.guid.new(),
        RelatedObjects=[element],
        RelatingPropertyDefinition=pset
    )
    ifc.add(pset)
    ifc.add(rel)

# 1. Créer le fichier IFC
ifc = ifcopenshell.file()

# 2. Projet
project = ifc.create_entity("IfcProject", GlobalId=ifcopenshell.guid.new(), Name="Projet Salle de formation")
ifc.add(project)

# 3. Site (géolocalisation)
site = ifc.create_entity("IfcSite", GlobalId=ifcopenshell.guid.new(), Name="Site ENSMR")
site.RefLatitude = [33, 59, 0, 0]   # Exemple : 33°59'N
site.RefLongitude = [-6, 51, 0, 0]  # Exemple : 6°51'W
site.RefElevation = 40.0
ifc.add(site)

# 4. Unités SI
unit_assignment = ifc.create_entity("IfcUnitAssignment", Units=[
    ifc.create_entity("IfcSIUnit", UnitType="LENGTHUNIT", Prefix="MILLI", Name="METRE"),
    ifc.create_entity("IfcSIUnit", UnitType="THERMODYNAMICTEMPERATUREUNIT", Name="DEGREE_CELSIUS"),
    ifc.create_entity("IfcSIUnit", UnitType="ENERGYUNIT", Name="JOULE"),
    ifc.create_entity("IfcSIUnit", UnitType="POWERUNIT", Name="WATT"),
    ifc.create_entity("IfcSIUnit", UnitType="LUMINOUSFLUXUNIT", Name="LUMEN"),
    ifc.create_entity("IfcSIUnit", UnitType="VOLUMEUNIT", Name="CUBIC_METRE")
])
project.UnitsInContext = unit_assignment

# 5. Salle (IfcSpace) avec dimensions et usage
room = ifc.create_entity("IfcSpace", GlobalId=ifcopenshell.guid.new(), Name="Salle de formation")
ifc.add(room)
pset_room = {
    "Longueur": 8.0,
    "Largeur": 6.0,
    "Hauteur": 3.0,
    "Surface": 48.0,
    "Volume": 144.0,
    "OccupationMaximale": 20,
    "TypeUsage": "Salle de cours",
    "TauxOccupation": 0.75
}
add_property_set(ifc, room, "PSet_Salle", pset_room)

# 6. Murs (IfcWall) avec propriétés thermiques
walls = []
for i in range(4):
    wall = ifc.create_entity("IfcWall", GlobalId=ifcopenshell.guid.new(), Name=f"Mur {i+1}")
    ifc.add(wall)
    pset_wall = {
        "Epaisseur": 0.3,
        "Materiau": "Béton",
        "ResistanceThermique": 2.1,  # (m²·K)/W valeur estimée pour Rabat
        "Conductivite": 1.4,  # W/(m·K) pour le béton
        "TransmittanceLumineuse": 0.0  # opaque
    }
    add_property_set(ifc, wall, "PSet_Mur", pset_wall)
    rel_containment = ifc.create_entity("IfcRelContainedInSpatialStructure",
        GlobalId=ifcopenshell.guid.new(),
        RelatingStructure=room,
        RelatedElements=[wall]
    )
    ifc.add(rel_containment)
    walls.append(wall)

# 7. Équipement CVC
cvc = ifc.create_entity("IfcFlowTerminal", GlobalId=ifcopenshell.guid.new(), Name="Climatiseur")
ifc.add(cvc)
pset_cvc = {
    "Puissance thermique": 4.5,
    "Conso energie": 2.3,
    "Mode fonctionnement": "Refroidissement",
    "Date mise en service": "2023-09-01"
}
add_property_set(ifc, cvc, "PSet_CVC", pset_cvc)

# 8. Éclairage
light = ifc.create_entity("IfcLightFixture", GlobalId=ifcopenshell.guid.new(), Name="Lumière LED")
ifc.add(light)
pset_light = {
    "Intensite nominale": 1200,
    "Luminosite mesuree": 450.0,
    "Conso energie": 0.25,
    "Type eclairage": "LED",
    "TauxOccupationCible": 0.8
}
add_property_set(ifc, light, "PSet_Eclairage", pset_light)

# 9. Capteurs
donnees_path = "minio://raw/capteurs/environnement/donnees_capteurs_20250705.csv"
sensors = {
    "Température": {
        "Temperature": 22.7,
        "DataSource": donnees_path
    },
    "Humidité": {
        "Taux humidite": 45.0,
        "DataSource": donnees_path
    },
    "CO2": {
        "Concentration": 520,
        "DataSource": donnees_path
    },
    "Présence": {
        "Occupant": 1,
        "DataSource": donnees_path
    }
}
sensor_entities = []
for name, props in sensors.items():
    sensor = ifc.create_entity("IfcSensor", GlobalId=ifcopenshell.guid.new(), Name=f"Capteur {name}")
    ifc.add(sensor)
    add_property_set(ifc, sensor, f"PSet_Capteur_{name}", props)
    sensor_entities.append(sensor)

# 10. Systèmes
sys_cvc = ifc.create_entity("IfcSystem", GlobalId=ifcopenshell.guid.new(), Name="Système CVC")
ifc.add(sys_cvc)
sys_light = ifc.create_entity("IfcSystem", GlobalId=ifcopenshell.guid.new(), Name="Système éclairage")
ifc.add(sys_light)
sys_env = ifc.create_entity("IfcSystem", GlobalId=ifcopenshell.guid.new(), Name="Système environnement")
ifc.add(sys_env)

# 11. Associations spatiales
for obj in [cvc, light] + sensor_entities:
    rel_containment = ifc.create_entity("IfcRelContainedInSpatialStructure",
        GlobalId=ifcopenshell.guid.new(),
        RelatingStructure=room,
        RelatedElements=[obj]
    )
    ifc.add(rel_containment)

# 12. Associations systèmes
ifc.add(ifc.create_entity("IfcRelAssignsToGroup", GlobalId=ifcopenshell.guid.new(), RelatingGroup=sys_cvc, RelatedObjects=[cvc]))
ifc.add(ifc.create_entity("IfcRelAssignsToGroup", GlobalId=ifcopenshell.guid.new(), RelatingGroup=sys_light, RelatedObjects=[light]))
for sensor in sensor_entities:
    ifc.add(ifc.create_entity("IfcRelAssignsToGroup", GlobalId=ifcopenshell.guid.new(), RelatingGroup=sys_env, RelatedObjects=[sensor]))

# 13. Sauvegarde
ifc.write("/home/amina/Bureau/salle_de_formation_complet.ifc")
print("✅ Fichier IFC enrichi avec propriétés murales, capteurs, MinIO et systèmes généré avec succès.")

✅ Fichier IFC enrichi avec propriétés murales, capteurs, MinIO et systèmes généré avec succès.


In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, sys
import argparse
import ifcopenshell, ifcopenshell.api, ifcopenshell.guid as ifcguid
from pathlib import Path

# ===== MinIO/S3 =====
def get_s3_client(endpoint, access, secret, secure):
    import boto3
    from botocore.config import Config
    return boto3.client(
        "s3",
        endpoint_url=endpoint,
        aws_access_key_id=access,
        aws_secret_access_key=secret,
        use_ssl=bool(secure),
        verify=bool(secure),
        region_name="us-east-1",
        config=Config(signature_version="s3v4"),
    )

def ensure_bucket(s3, bucket):
    import botocore
    try: s3.head_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError: s3.create_bucket(Bucket=bucket)

def s3_upload_file(s3, bucket, local_path:Path, key:str):
    s3.upload_file(str(local_path), bucket, key)

# ===== IFC helpers =====
def add_pset(m, el, name, props):
    pset = ifcopenshell.api.run("pset.add_pset", m, product=el, name=name)
    ifcopenshell.api.run("pset.edit_pset", m, pset=pset, properties=props)

def local_placement(m, relative_to=None, x=0.0, y=0.0, z=0.0):
    cp = m.create_entity("IfcCartesianPoint", Coordinates=[float(x), float(y), float(z)])
    ax = m.create_entity("IfcAxis2Placement3D", Location=cp)
    return m.create_entity("IfcLocalPlacement", PlacementRelTo=relative_to, RelativePlacement=ax)

# ===== Build IFC =====
def build_ifc(month:str, out_path:Path)->None:
    ifc = ifcopenshell.file(schema="IFC4")

    # units + context
    units = [
        ifc.create_entity("IfcSIUnit", UnitType="LENGTHUNIT", Prefix="MILLI", Name="METRE"),
        ifc.create_entity("IfcSIUnit", UnitType="THERMODYNAMICTEMPERATUREUNIT", Name="DEGREE_CELSIUS"),
        ifc.create_entity("IfcSIUnit", UnitType="POWERUNIT", Name="WATT"),
        ifc.create_entity("IfcSIUnit", UnitType="ENERGYUNIT", Name="JOULE"),
        ifc.create_entity("IfcSIUnit", UnitType="LUMINOUSFLUXUNIT", Name="LUMEN"),
        ifc.create_entity("IfcSIUnit", UnitType="VOLUMEUNIT", Name="CUBIC_METRE"),
    ]
    ua = ifc.create_entity("IfcUnitAssignment", Units=units)
    wcs = ifc.create_entity("IfcAxis2Placement3D",
            Location=ifc.create_entity("IfcCartesianPoint", Coordinates=[0.0,0.0,0.0]))
    ctx = ifc.create_entity("IfcGeometricRepresentationContext",
            ContextIdentifier="Model", ContextType="Model",
            CoordinateSpaceDimension=3, Precision=1e-5, WorldCoordinateSystem=wcs)

    project = ifc.create_entity("IfcProject", GlobalId=ifcguid.new(), Name="Training Room Project",
                                UnitsInContext=ua, RepresentationContexts=[ctx]); ifc.add(project)

    site = ifc.create_entity("IfcSite", GlobalId=ifcguid.new(), Name="ENSMR Site",
                             ObjectPlacement=local_placement(ifc), CompositionType="ELEMENT")
    site.RefLatitude=[33,59,0,0]; site.RefLongitude=[-6,51,0,0]; site.RefElevation=40.0; ifc.add(site)
    building = ifc.create_entity("IfcBuilding", GlobalId=ifcguid.new(), Name="Teaching Building",
                                 ObjectPlacement=local_placement(ifc, site.ObjectPlacement), CompositionType="ELEMENT"); ifc.add(building)
    storey = ifc.create_entity("IfcBuildingStorey", GlobalId=ifcguid.new(), Name="Ground Floor",
                               Elevation=0.0, ObjectPlacement=local_placement(ifc, building.ObjectPlacement),
                               CompositionType="ELEMENT"); ifc.add(storey)
    ifc.add(ifc.create_entity("IfcRelAggregates", GlobalId=ifcguid.new(), RelatingObject=project, RelatedObjects=[site]))
    ifc.add(ifc.create_entity("IfcRelAggregates", GlobalId=ifcguid.new(), RelatingObject=site, RelatedObjects=[building]))
    ifc.add(ifc.create_entity("IfcRelAggregates", GlobalId=ifcguid.new(), RelatingObject=building, RelatedObjects=[storey]))

    room = ifc.create_entity("IfcSpace", GlobalId=ifcguid.new(), Name="Training Room 101",
                             ObjectPlacement=local_placement(ifc, storey.ObjectPlacement), CompositionType="ELEMENT"); ifc.add(room)
    add_pset(ifc, room, "PSet_TrainingRoom",
             {"Length_m":8.0,"Width_m":6.0,"Height_m":3.0,"Area_m2":48.0,"Volume_m3":144.0,
              "MaxOccupancy":20,"UseType":"Classroom","TargetOccupancyRate":0.75})
    ifc.add(ifc.create_entity("IfcRelContainedInSpatialStructure",
                              GlobalId=ifcguid.new(), RelatingStructure=storey, RelatedElements=[room]))

    for i in range(4):
        wall = ifc.create_entity("IfcWall", GlobalId=ifcguid.new(), Name=f"Wall_{i+1}",
                                 ObjectPlacement=local_placement(ifc, room.ObjectPlacement)); ifc.add(wall)
        add_pset(ifc, wall, "PSet_Wall_Thermal",
                 {"Thickness_m":0.30,"Material":"Concrete","R_Value_m2K_W":2.1,"Conductivity_W_mK":1.4,"VisibleTransmittance":0.0})
        ifc.add(ifc.create_entity("IfcRelContainedInSpatialStructure",
                                  GlobalId=ifcguid.new(), RelatingStructure=room, RelatedElements=[wall]))

    hvac = ifc.create_entity("IfcFlowTerminal", GlobalId=ifcguid.new(), Name="AC_Unit_1",
                             ObjectPlacement=local_placement(ifc, room.ObjectPlacement)); ifc.add(hvac)
    add_pset(ifc, hvac, "PSet_HVAC",
             {"ThermalPower_kW":4.5,"ElectricalPower_kW":2.3,"Mode":"Cooling","CommissioningDate":"2023-09-01"})

    light = ifc.create_entity("IfcLightFixture", GlobalId=ifcguid.new(), Name="LED_Luminaire_1",
                              ObjectPlacement=local_placement(ifc, room.ObjectPlacement)); ifc.add(light)
    add_pset(ifc, light, "PSet_Lighting",
             {"RatedFlux_lm":1200,"MeasuredIlluminance_lux":450.0,"ElectricPower_kW":0.25,"Type":"LED"})

    minio_prefix = "minio://raw"
    for name, props in {
        "Temp_Sensor_1":{"Temperature_C":22.7,"DataSource":f"{minio_prefix}/sensors/{month}/zone_101_sensors.csv"},
        "Humidity_Sensor_1":{"RH_pct":45.0,"DataSource":f"{minio_prefix}/sensors/{month}/zone_101_sensors.csv"},
        "CO2_Sensor_1":{"CO2_ppm":520,"DataSource":f"{minio_prefix}/sensors/{month}/zone_101_sensors.csv"},
        "Occupancy_Sensor_1":{"Occupied":1,"DataSource":f"{minio_prefix}/occupancy/{month}/occupancy.csv"},
    }.items():
        s = ifc.create_entity("IfcSensor", GlobalId=ifcguid.new(), Name=name,
                              ObjectPlacement=local_placement(ifc, room.ObjectPlacement)); ifc.add(s)
        add_pset(ifc, s, f"PSet_{name}", props)
        ifc.add(ifc.create_entity("IfcRelContainedInSpatialStructure",
                                  GlobalId=ifcguid.new(), RelatingStructure=room, RelatedElements=[s]))

    ifc.write(str(out_path))

# ===== Main =====
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--month", type=str, default="2025-03")  # YYYY-MM
    ap.add_argument("--outdir", type=str, default="~/DTE/jne_project/raw/ifc")
    # MinIO
    ap.add_argument("--endpoint", type=str, default=os.environ.get("MINIO_ENDPOINT","http://192.168.0.173:9000"))
    ap.add_argument("--access",   type=str, default=os.environ.get("MINIO_ROOT_USER","minioadmin"))
    ap.add_argument("--secret",   type=str, default=os.environ.get("MINIO_ROOT_PASSWORD","minioadmin"))
    ap.add_argument("--bucket",   type=str, default="raw")
    ap.add_argument("--prefix",   type=str, default="jne_project/raw")
    ap.add_argument("--secure",   action="store_true")
    ap.add_argument("--no-upload", action="store_true")
    args,_ = ap.parse_known_args()

    out_dir = Path(args.outdir).expanduser().resolve()/args.month
    out_dir.mkdir(parents=True, exist_ok=True)
    local_ifc = out_dir/"training_room.ifc"

    build_ifc(args.month, local_ifc)

    if not args.no_upload:
        try:
            s3 = get_s3_client(args.endpoint, args.access, args.secret, args.secure)
            ensure_bucket(s3, args.bucket)
            root = args.prefix.strip("/")
            key = f"{root}/ifc/{args.month}/training_room.ifc"
            s3_upload_file(s3, args.bucket, local_ifc, key)
            print("minio:", f"s3://{args.bucket}/{key}")
        except Exception as e:
            print("ERREUR upload MinIO:", e, file=sys.stderr); sys.exit(3)

    print("local:", local_ifc)

if __name__ == "__main__":
    main()


minio: s3://raw/jne_project/raw/ifc/2025-03/training_room.ifc
local: /home/amina/DTE/jne_project/raw/ifc/2025-03/training_room.ifc


minio: s3://refined/jne_project/refined/brick/2025-03/training_room.ttl
local: /home/amina/DTE/jne_project/refined/brick/2025-03/training_room.ttl | /home/amina/DTE/jne_project/refined/meta/2025-03/brick_manifest.json
